<a href="https://colab.research.google.com/github/RohanOpenSource/Deep-Learning-And-Beyond/blob/main/ExplodingAndVanishingGradients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training neural networks can come with a host of problems that are tricky to combat. The first of these is the exploding and vanishing gradient problem. During backpropogation, gradients can either become progressively smaller, leaving the lower layers of the neural network to remain unchanged, or they can become larger making the neural network not work at all. To have this not happen, we need a similar variance for the inputs of each layer and the outputs. To acheive this, we can randomly initialize the weights with some constraints. This is the basis of Glorot, He, and LeCun intilialization and is what is used by default in Keras.

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import load_iris

In [2]:
iris = load_iris() #you know the drill we're using iris
X =  iris.data[:, 2:]
y = iris.target

In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation="sigmoid", kernel_initializer=tf.keras.initializers.GlorotUniform), 
    tf.keras.layers.Dense(64, activation="sigmoid", kernel_initializer=tf.keras.initializers.GlorotUniform), 
    tf.keras.layers.Dense(64, activation="sigmoid", kernel_initializer=tf.keras.initializers.GlorotUniform), 
    tf.keras.layers.Dense(64, activation="sigmoid", kernel_initializer=tf.keras.initializers.GlorotUniform), 
    tf.keras.layers.Dense(64, activation="sigmoid", kernel_initializer=tf.keras.initializers.GlorotUniform), 
    tf.keras.layers.Dense(64, activation="sigmoid", kernel_initializer=tf.keras.initializers.GlorotUniform), 
    tf.keras.layers.Dense(3, activation="softmax") 
])
model.build([1, 2])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (1, 128)                  384       
                                                                 
 dense_1 (Dense)             (1, 64)                   8256      
                                                                 
 dense_2 (Dense)             (1, 64)                   4160      
                                                                 
 dense_3 (Dense)             (1, 64)                   4160      
                                                                 
 dense_4 (Dense)             (1, 64)                   4160      
                                                                 
 dense_5 (Dense)             (1, 64)                   4160      
                                                                 
 dense_6 (Dense)             (1, 3)                    1

In [4]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(X, y, epochs=30)

Epoch 1/30
5/5 [==============================] - 2s 7ms/step - loss: 1.3203 - accuracy: 0.3333
Epoch 2/30
5/5 [==============================] - 0s 10ms/step - loss: 1.1270 - accuracy: 0.3333
Epoch 3/30
5/5 [==============================] - 0s 8ms/step - loss: 1.0992 - accuracy: 0.3600
Epoch 4/30
5/5 [==============================] - 0s 6ms/step - loss: 1.1142 - accuracy: 0.3333
Epoch 5/30
5/5 [==============================] - 0s 6ms/step - loss: 1.1167 - accuracy: 0.3333
Epoch 6/30
5/5 [==============================] - 0s 9ms/step - loss: 1.1096 - accuracy: 0.3067
Epoch 7/30
5/5 [==============================] - 0s 9ms/step - loss: 1.1046 - accuracy: 0.3333
Epoch 8/30
5/5 [==============================] - 0s 8ms/step - loss: 1.0989 - accuracy: 0.3467
Epoch 9/30
5/5 [==============================] - 0s 6ms/step - loss: 1.0994 - accuracy: 0.3333
Epoch 10/30
5/5 [==============================] - 0s 6ms/step - loss: 1.1003 - accuracy: 0.3333
Epoch 11/30
5/5 [=====================

As displayed above, changing the weight initlization is not enough to fix vanishing or exploding gradients. This is because activation functions can also be huge contributers to funky gradients.
 
Sigmoid can cause vanishing gradients because it squeezes the weights resulting in insanely small gradients during the end of backpropogation.

Relu isn't perfect either as it leads to some neurons always outputting 0, thus becoming obselete (just like pytorch).

Rather than making negative numbers 0, leaky relu squashes them quite heavily meaning no neurons become entirely useless (NO NEURONS LEFT BEHIND YAYYYY).

In [5]:
leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=leaky_relu, kernel_initializer="he_normal"), 
    tf.keras.layers.Dense(64, activation=leaky_relu, kernel_initializer="he_normal"), 
    tf.keras.layers.Dense(64, activation=leaky_relu, kernel_initializer="he_normal"), 
    tf.keras.layers.Dense(64, activation=leaky_relu, kernel_initializer="he_normal"), 
    tf.keras.layers.Dense(64, activation=leaky_relu, kernel_initializer="he_normal"), 
    tf.keras.layers.Dense(64, activation=leaky_relu, kernel_initializer="he_normal"), 
    tf.keras.layers.Dense(3, activation="softmax")
])


In [6]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(X, y, epochs=60)

Epoch 1/60
5/5 [==============================] - 1s 5ms/step - loss: 3.9875 - accuracy: 0.3133
Epoch 2/60
5/5 [==============================] - 0s 6ms/step - loss: 1.2109 - accuracy: 0.5600
Epoch 3/60
5/5 [==============================] - 0s 11ms/step - loss: 0.9194 - accuracy: 0.6200
Epoch 4/60
5/5 [==============================] - 0s 7ms/step - loss: 0.8432 - accuracy: 0.6600
Epoch 5/60
5/5 [==============================] - 0s 5ms/step - loss: 0.7465 - accuracy: 0.6867
Epoch 6/60
5/5 [==============================] - 0s 7ms/step - loss: 0.6870 - accuracy: 0.6533
Epoch 7/60
5/5 [==============================] - 0s 8ms/step - loss: 0.6196 - accuracy: 0.8400
Epoch 8/60
5/5 [==============================] - 0s 6ms/step - loss: 0.5674 - accuracy: 0.7467
Epoch 9/60
5/5 [==============================] - 0s 7ms/step - loss: 0.5053 - accuracy: 0.8333
Epoch 10/60
5/5 [==============================] - 0s 5ms/step - loss: 0.4880 - accuracy: 0.8400
Epoch 11/60
5/5 [=====================

Sometimes, EVEN ALL OF THIS NONSENSE IS NOT ENOUGH TO END THE WRATH OF THE FUNKY GRADIENT. In such cases, Batch Normalization normalizes, centers, shifts, and scales mini-batches of inputs of the layer before it allowing the model to learn the optimal scale for its data. This reduces the variance between layers and if used as the first layer, can remove the need to normalize the data before hand.

In [7]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X_valid, X_train = X_train_full[:5000] /  255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

4431872/4422102 [==============================] - 0s 0us/step


In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(50, activation="elu", kernel_initializer="he_normal"), #elu is like a combination of relu (x>=0) and an exponential function (x<0), its great but quite slow
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(50, activation="elu", kernel_initializer="he_normal"), 
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(50, activation="elu", kernel_initializer="he_normal"), 
    tf.keras.layers.Dense(10, activation="softmax")
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_14 (Dense)            (None, 50)                39250     
                                                                 
 batch_normalization_1 (Batc  (None, 50)               200       
 hNormalization)                                                 
                                                                 
 dense_15 (Dense)            (None, 50)                2550      
                                                                 
 batch_normalization_2 (Batc  (None, 50)              

In [9]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid)) # most resonable people would use more epochs but I've got romance novels to read so shut up

Epoch 1/10
1719/1719 [==============================] - 8s 4ms/step - loss: 0.4958 - accuracy: 0.8225 - val_loss: 0.3737 - val_accuracy: 0.8670
Epoch 2/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.3819 - accuracy: 0.8601 - val_loss: 0.3459 - val_accuracy: 0.8746
Epoch 3/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3485 - accuracy: 0.8715 - val_loss: 0.3189 - val_accuracy: 0.8854
Epoch 4/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3273 - accuracy: 0.8788 - val_loss: 0.3056 - val_accuracy: 0.8888
Epoch 5/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.3104 - accuracy: 0.8846 - val_loss: 0.3211 - val_accuracy: 0.8862
Epoch 6/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2998 - accuracy: 0.8881 - val_loss: 0.3201 - val_accuracy: 0.8834
Epoch 7/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2884 - accuracy: 0.8933 - val_loss: 0.2969 - val_accuracy:

Now what would be the fun

One can avoid exploding gradients by clippping the gradients before they become too big, during backpropogation. This is called gradient clipping and is quite effective and easy to use. 

In [10]:
optimizer = tf.keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss = "sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.fit(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 6s 3ms/step - loss: 0.2260 - accuracy: 0.9155 - val_loss: 0.2897 - val_accuracy: 0.8982
Epoch 2/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2207 - accuracy: 0.9177 - val_loss: 0.2904 - val_accuracy: 0.8968
Epoch 3/5
1719/1719 [==============================] - 6s 4ms/step - loss: 0.2173 - accuracy: 0.9183 - val_loss: 0.2879 - val_accuracy: 0.8974
Epoch 4/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2133 - accuracy: 0.9203 - val_loss: 0.2857 - val_accuracy: 0.8976
Epoch 5/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.2131 - accuracy: 0.9202 - val_loss: 0.2898 - val_accuracy: 0.8942


That's all for this notebook. I'm eternally greatful to finally be back in the game after a year long hiatus caused by school and poor time management. I realized that pursuing coding is a possible side activity and is more important to me than doing well in school(even though I am and will probably continue to).